In [29]:
#Dependencies
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [30]:
#read csv into df
df = pd.read_csv("https://data-finalproject.s3.amazonaws.com/Levels_Fyi_Salary_Data.csv")
# columns to drop... salary_currency, salary, work_year
df.sample(25)


,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
41740,2/10/2021 21:57:39,Flipkart,SDE 1,Software Engineer,31000,"Bangalore, KA, India",4.0,2.0,Distributed Systems (Back-End),26000.0,...,0,0,0,1,0,0,0,0,Asian,Bachelor's Degree
45000,3/14/2021 23:39:33,Cisco,Software Engineer III,Software Engineer,29000,"Bangalore, KA, India",5.0,5.0,Distributed Systems (Back-End),22000.0,...,0,0,0,0,0,0,0,0,NaN,Master's Degree
18557,3/21/2020 6:25:23,Amazon,L4,Software Engineer,20000,"Bangalore, KA, India",2.0,2.0,Distributed Systems (Back-End),18000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
5734,4/12/2019 14:10:27,Tesla,Senior Engineer,Software Engineer,200000,"Palo Alto, CA",4.0,4.0,Distributed Systems (Back-End),150000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
55822,6/27/2021 23:05:46,Facebook,IC4,Data Scientist,226000,"Fremont, CA",10.0,2.0,Infra,160000.0,...,0,0,0,0,1,0,0,0,White,Master's Degree
412,6/4/2018 20:27:58,Splunk,P2,Software Engineer,160000,"Seattle, WA",5.0,3.0,Full Stack,0.0,...,0,0,0,0,0,0,0,0,NaN,NaN
46505,3/29/2021 22:20:08,Google,L6,Software Engineer,409000,"Tel Aviv, TA, Israel",19.0,3.0,ML / AI,204000.0,...,0,0,0,0,1,0,0,0,White,Bachelor's Degree
47833,4/10/2021 23:27:34,Google,L5,Software Engineer,350000,"San Francisco, CA",5.0,5.0,Security,190000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
21043,5/15/2020 18:39:34,Apple,ICT4,Software Engineer,415000,"Cupertino, CA",12.0,3.0,ML / AI,205000.0,...,1,0,0,0,0,0,0,0,NaN,PhD
46674,3/31/2021 15:07:16,Expedia,K,Data Scientist,136000,"Austin, TX",0.0,0.0,General,115000.0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [31]:
df.nunique()


timestamp                  62561
company                     1631
level                       2923
title                         15
totalyearlycompensation      893
location                    1050
yearsofexperience             65
yearsatcompany                81
tag                         3058
basesalary                   482
stockgrantvalue              612
bonus                        335
gender                         4
otherdetails               12841
cityid                      1045
dmaid                        149
rowNumber                  62642
Masters_Degree                 2
Bachelors_Degree               2
Doctorate_Degree               2
Highschool                     2
Some_College                   2
Race_Asian                     2
Race_White                     2
Race_Two_Or_More               2
Race_Black                     2
Race_Hispanic                  2
Race                           5
Education                      5
dtype: int64

In [32]:
titles_df = df[df["title"]=="Data Scientist"]
titles_df


,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
419,6/5/2018 14:06:30,LinkedIn,Senior,Data Scientist,233000,"San Francisco, CA",4.0,0.0,Data Analysis,162000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
440,6/8/2018 9:49:25,Microsoft,64,Data Scientist,218000,"Seattle, WA",11.0,11.0,ML / AI,165000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
444,6/8/2018 17:55:09,ebay,26,Data Scientist,180000,"San Jose, CA",10.0,5.0,NaN,0.0,...,0,0,0,0,0,0,0,0,NaN,NaN
454,6/10/2018 19:39:35,Twitter,Staff,Data Scientist,500000,"San Francisco, CA",4.0,4.0,ML / AI,200000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
495,6/17/2018 11:39:38,Facebook,5,Data Scientist,370000,"Seattle, WA",8.0,3.0,NaN,190000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62240,3/26/2019 17:25:02,IBM,8,Data Scientist,155000,"Washington, DC",5.0,2.0,ML / AI,141000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
62283,6/17/2018 21:07:18,Facebook,E3,Data Scientist,150000,"Singapore, SG, Singapore",0.0,0.0,ML / AI,150000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
62285,7/16/2018 20:11:40,ADP,Sr UX Researcher 4,Data Scientist,185000,"Roseland, NJ",8.0,2.0,UX Research,150000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
62529,6/12/2018 20:54:06,Google,T6,Data Scientist,685000,"Kirkland, WA",22.0,2.0,ML / AI,221000.0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [33]:
# df1 = titles_df.copy()
# filtered_columns = ["company", "location", "yearsofexperience", "totalyearlycompensation",
#                  "Masters_Degree", "Bachelors_Degree"]
# df2 = df1[filtered_columns]
# # print(df1.shape)
# df2.head()

In [34]:
clean_df= titles_df.drop(columns=['timestamp', 'level', 'yearsatcompany', 'otherdetails','Highschool', 'Some_College', 
                               'Race_Asian', 'Race_White', 'Race_Two_Or_More', 'Race_Black',"gender",
                              'Race_Hispanic', 'Race', 'Education','cityid','dmaid','rowNumber','tag','bonus','stockgrantvalue','basesalary','title'])
clean_df.head()
#get_dummies/oneHotEncoder on company size and employment type

,company,totalyearlycompensation,location,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree
419,LinkedIn,233000,"San Francisco, CA",4.0,0,0,0
440,Microsoft,218000,"Seattle, WA",11.0,0,0,0
444,ebay,180000,"San Jose, CA",10.0,0,0,0
454,Twitter,500000,"San Francisco, CA",4.0,0,0,0
495,Facebook,370000,"Seattle, WA",8.0,0,0,0


In [35]:
import re
form_state_one = r'(\w+\s)*(\w+),\s([A-Z]{2})$'
matches_form_one = clean_df.location.str.contains(form_state_one, flags=re.IGNORECASE, na=False)
no_form_one = clean_df.location.str.contains(form_state_one, flags=re.IGNORECASE, na=False).sum()
print(no_form_one)

form_state_two = r'(\w+\s)*(\w+),\s([A-Z]{2}),\s(\w+)'
matches_form_two = clean_df.location.str.contains(form_state_two, flags=re.IGNORECASE, na=False)
no_form_two = clean_df.location.str.contains(form_state_two, flags=re.IGNORECASE, na=False).sum()
print(no_form_two)


clean_df.location[~matches_form_one & ~matches_form_two]

2226
351


C:\Users\siriu\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\siriu\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  after removing the cwd from sys.path.
C:\Users\siriu\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
C:\Users\siriu\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:9: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  if __name__ == '__main__':


9061    Tel Aviv, Israel
Name: location, dtype: object

In [36]:
clean_df.head()

,company,totalyearlycompensation,location,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree
419,LinkedIn,233000,"San Francisco, CA",4.0,0,0,0
440,Microsoft,218000,"Seattle, WA",11.0,0,0,0
444,ebay,180000,"San Jose, CA",10.0,0,0,0
454,Twitter,500000,"San Francisco, CA",4.0,0,0,0
495,Facebook,370000,"Seattle, WA",8.0,0,0,0


In [37]:
def US_location_to_state(city):
    # if input is of the form $###,###,###
    if re.match(form_state_one, city, flags=re.IGNORECASE):

        # remove city name and commas
        state = re.sub('(\w+\s)*(\w+),\s','', city)
        
        # return state
        return state
    # otherwise, return NaN
    else:
        return np.nan
    

def location_to_city(city):
    # if input is of the form $###,###,###
    if re.match(form_state_one, city, flags=re.IGNORECASE):

        # remove city name and commas
        city = re.sub(',\s([A-Z]{2})$','', city)
        
        # return state
        return city
    # otherwise, return NaN
    else:
        return np.nan
clean_df = clean_df.loc[clean_df.location != "Tel Aviv, Israel"]
clean_df


,company,totalyearlycompensation,location,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree
419,LinkedIn,233000,"San Francisco, CA",4.0,0,0,0
440,Microsoft,218000,"Seattle, WA",11.0,0,0,0
444,ebay,180000,"San Jose, CA",10.0,0,0,0
454,Twitter,500000,"San Francisco, CA",4.0,0,0,0
495,Facebook,370000,"Seattle, WA",8.0,0,0,0
...,...,...,...,...,...,...,...
62240,IBM,155000,"Washington, DC",5.0,0,0,0
62283,Facebook,150000,"Singapore, SG, Singapore",0.0,0,0,0
62285,ADP,185000,"Roseland, NJ",8.0,0,0,0
62529,Google,685000,"Kirkland, WA",22.0,0,0,0


In [38]:
clean_df['state'] = clean_df.location.str.extract(f'({form_state_one}|{form_state_two})', flags=re.IGNORECASE)[0].apply(US_location_to_state)

In [39]:
clean_df = clean_df[clean_df['state'].notna()]
clean_df.head()

,company,totalyearlycompensation,location,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree,state
419,LinkedIn,233000,"San Francisco, CA",4.0,0,0,0,CA
440,Microsoft,218000,"Seattle, WA",11.0,0,0,0,WA
444,ebay,180000,"San Jose, CA",10.0,0,0,0,CA
454,Twitter,500000,"San Francisco, CA",4.0,0,0,0,CA
495,Facebook,370000,"Seattle, WA",8.0,0,0,0,WA


In [40]:
clean_df['city'] = clean_df.location.str.extract(f'({form_state_one})', flags=re.IGNORECASE)[0].apply(location_to_city)
clean_df.drop('location', axis=1, inplace=True)
clean_df.drop('city', axis=1, inplace=True)
clean_df.head()

C:\Users\siriu\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\siriu\anaconda3\envs\mlenv\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,company,totalyearlycompensation,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree,state
419,LinkedIn,233000,4.0,0,0,0,CA
440,Microsoft,218000,11.0,0,0,0,WA
444,ebay,180000,10.0,0,0,0,CA
454,Twitter,500000,4.0,0,0,0,CA
495,Facebook,370000,8.0,0,0,0,WA


In [41]:
print(clean_df["company"].value_counts().head(10))

Amazon          224
Microsoft       202
Facebook        175
Google           83
Apple            75
Uber             60
IBM              58
Capital One      44
LinkedIn         34
Walmart Labs     33
Name: company, dtype: int64


In [42]:
# #Binning he companies- .cut with 4 bins...do we want to specifically design these?
# bin_labels = ['Small', 'Medium', 'Large']
# bins3 = clean_df['company'].value_counts()
# company_bins = pd.cut(bins3, labels=bin_labels).to_list()#does this solve the issue of imbalanced data?
# company_bins

In [43]:
#Binning he companies- .cut with 4 bins...do we want to specifically design these?
bin_labels = ['Small', 'Medium', 'Large', 'Very Large']
bins4 = clean_df['company'].value_counts()
company_bins = pd.cut(bins4, bins=4, labels=bin_labels).to_list()#does this solve the issue of imbalanced data?
company_bins


['Very Large',
 'Very Large',
 'Very Large',
 'Medium',
 'Medium',
 'Medium',
 'Medium',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 'Small',
 

In [44]:
#change company name to bin name
companies = []
for company in clean_df['company']:
    if company == "Amazon" or company == "Microsoft" or company == "Facebook":
        company = "Large"
    elif company == "Google" or company == "Apple" or company == "IBM" or company=="Uber":
        company = "Medium"
    else:
        company = "Small"
    companies.append(company)

In [45]:
#reassign company column to bin name
clean_bins_df = clean_df.copy()
clean_bins_df['company'] = companies
clean_bins_df.head(10)
# clean_bins_df['gender'].value_counts()

,company,totalyearlycompensation,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree,state
419,Small,233000,4.0,0,0,0,CA
440,Large,218000,11.0,0,0,0,WA
444,Small,180000,10.0,0,0,0,CA
454,Small,500000,4.0,0,0,0,CA
495,Large,370000,8.0,0,0,0,WA
499,Large,200000,3.0,0,0,0,WA
509,Large,340000,11.0,0,0,0,WA
510,Medium,690000,10.0,0,0,0,WA
513,Small,600000,3.0,0,0,0,CA
523,Small,168000,8.0,0,0,0,CA


In [46]:
clean_dummies = pd.get_dummies(data=clean_bins_df, columns=['company', 'state'])#drop the gender columns?
clean_dummies.head(15)

,totalyearlycompensation,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree,company_Large,company_Medium,company_Small,state_AL,state_AR,...,state_OR,state_PA,state_RI,state_SC,state_TN,state_TX,state_UT,state_VA,state_WA,state_WI
419,233000,4.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
440,218000,11.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
444,180000,10.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
454,500000,4.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
495,370000,8.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
499,200000,3.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
509,340000,11.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
510,690000,10.0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
513,600000,3.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
523,168000,8.0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
clean_dummies.corr(method ='kendall')

,totalyearlycompensation,yearsofexperience,Masters_Degree,Bachelors_Degree,Doctorate_Degree,company_Large,company_Medium,company_Small,state_AL,state_AR,...,state_OR,state_PA,state_RI,state_SC,state_TN,state_TX,state_UT,state_VA,state_WA,state_WI
totalyearlycompensation,1.000000,0.294166,-0.069762,-0.095127,0.170405,0.171009,0.099680,-0.222606,-0.017385,0.000062,...,-0.029886,-0.054373,-0.026613,-0.007531,-0.038135,-0.138639,-0.030524,-0.118682,0.072512,-0.027560
yearsofexperience,0.294166,1.000000,0.026833,-0.025244,-0.022686,0.028119,0.018678,-0.038146,0.039918,0.055414,...,0.024439,-0.003060,-0.000430,0.026414,0.017126,-0.029778,-0.037067,-0.005570,0.046376,-0.003099
Masters_Degree,-0.069762,0.026833,1.000000,-0.188774,-0.238420,-0.039257,-0.027079,0.053932,0.002888,0.047693,...,-0.047280,-0.038999,0.013292,0.032585,0.029677,0.046768,-0.027604,0.005897,-0.050967,0.013292
Bachelors_Degree,-0.095127,-0.025244,-0.188774,1.000000,-0.126861,-0.023549,0.024090,0.005147,-0.010863,-0.002629,...,0.045744,0.008479,-0.008868,-0.006269,0.034168,0.066049,0.065467,0.057783,-0.033673,-0.008868
Doctorate_Degree,0.170405,-0.022686,-0.238420,-0.126861,1.000000,0.021178,0.045521,-0.049945,0.051834,-0.034130,...,0.080750,0.014108,-0.012865,-0.009095,-0.015760,-0.037016,-0.018202,0.005783,0.003254,0.028518
company_Large,0.171009,0.028119,-0.039257,-0.023549,0.021178,1.000000,-0.228796,-0.754252,-0.022341,-0.048382,...,-0.074602,-0.023428,-0.018237,-0.012893,0.005241,-0.091159,-0.001911,-0.068255,0.560624,-0.018237
company_Medium,0.099680,0.018678,-0.027079,0.024090,0.045521,-0.228796,1.000000,-0.466599,-0.013821,-0.029930,...,-0.046150,-0.034907,-0.011282,0.056350,-0.013821,0.014245,-0.015962,-0.041398,-0.131675,-0.011282
company_Small,-0.222606,-0.038146,0.053932,0.005147,-0.049945,-0.754252,-0.466599,1.000000,0.029620,0.064145,...,0.098908,0.044830,0.024179,-0.026293,0.004560,0.073217,0.012502,0.089936,-0.420554,0.024179
state_AL,-0.017385,0.039918,0.002888,-0.010863,0.051834,-0.022341,-0.013821,0.029620,1.000000,-0.002923,...,-0.004506,-0.003409,-0.001102,-0.000779,-0.001350,-0.007139,-0.001559,-0.005032,-0.018492,-0.001102
state_AR,0.000062,0.055414,0.047693,-0.002629,-0.034130,-0.048382,-0.029930,0.064145,-0.002923,1.000000,...,-0.009759,-0.007382,-0.002386,-0.001687,-0.002923,-0.015460,-0.003375,-0.010898,-0.040046,-0.002386


In [ ]:
clean_bins_df.corr(method ='kendall')

In [47]:
clean_dummies.to_csv("Cleaned_dataset.csv", sep=',', header=True, encoding='utf-8', index=False)

In [19]:
# #is there linearity between location numerical representation and salary?
# clean_bins2 = clean_bins_df.drop(['location'], axis = 1)
# locations = clean_bins_df['location']
# clean_bins2.head()

In [ ]:
# #Create label Encoder and encode categorical
# from sklearn.preprocessing import LabelEncoder
# lb_make = LabelEncoder()
# clean_bins2['company']= lb_make.fit_transform(clean_bins2['company'])
# clean_bins2['gender']= lb_make.fit_transform(clean_bins2['gender'])

# clean_bins2.head()

In [ ]:
# clean_bins2['gender'].value_counts() #what do we do with 4 genders?